<a href="https://colab.research.google.com/github/chirayu-xx/pdf-summarizer/blob/main/pdf_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install transformers
!pip install pytesseract
!pip install PyMuPDF
!pip install sentencepiece

In [20]:
from bs4 import BeautifulSoup

In [25]:
path = '/content/Chirayu_Shah_Resume.pdf'
doc = fitz.open(path)
text = ""
for page_number in range(doc.page_count):
  page = doc[page_number]
  text += page.get_text()

soup = BeautifulSoup(text, 'html.parser')
text = soup.get_text(separator= ' ')

In [26]:
text

'Chirayu Shah\nLocation: Chandigarh, India\nSoftware Developer\nPortfolio | LinkedIn | GitHub | Leetcode\nEmail: chirayu.shah2103@gmail.com | Mobile: 9586700639\nSUMMARY\nDedicated software developer with adept problem-solving skills and firm grasp of system design. Proficient in Re-\nact.js, NEXT.js, and Node.js frameworks, I excel in creating innovative solutions. A skilled team player, I contribute\nharmoniously to collective endeavors.\nTECHNICAL SKILLS\nLanguages: C++, Javascript,Python,HTML,CSS\nFrameworks: ReactJS, TailwindCSS, NEXT.JS, NodeJS,MongoDB,ExpressJS\nCertifications: React: Design Patterns,Node JS Essential\nOthers: Data Structures and Algorithm , System Design ,Collabaration Skills , Management Skills\nEXPERIENCE\nGruby Pvt Ltd, Remote: Full Stack Developer\nDec 2022 – Jun 2023\n• Delivered top-notch software solutions to a dynamic Saas Startup, spearheading the development and continual\nenhancement of novel features across web portals.\n• Orchestrated seamless team

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer=AutoTokenizer.from_pretrained('T5-base')
model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)

# Encode the text
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt",max_length=512, truncation=True)

# Generate the summary
outputs = model.generate(inputs,
max_length=1000, min_length=100)

# Decode the summary
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)


Dedicated software developer with adept problem-solving skills. excels in creating innovative solutions. a skilled team player, i contribute harmoniously to collective endeavors. delivered top-notch software solutions to a dynamic Saas Startup. a frontend developer at Codbrix Lab. a YouTube 2.0 winter 2022 project. a.. a.. a.. a.. a.. a.. 


In [33]:
summary

'<pad> Dedicated software developer with adept problem-solving skills and firm grasp of system design. proficient in React.js, NEXT.js, and Node.js frameworks, excel in creating innovative solutions. delivered top-notch software solutions to a dynamic Saas Startup, spearheading the development and continual enhancement of novel features across web portals. Designed and implemented an IMDb-like website using Next.js elevating UI/UX through innovative enhance- </s>'

In [ ]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn
!pip install python-multipart

In [ ]:
!pip install sentencepiece
!ngrok authtoken 2ZLDE06RzMLQEbRfbRmZzhHTdSJ_5vnBBVzitpreaSLRbFGxD

In [39]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, File, UploadFile,HTTPException
from fastapi.responses import JSONResponse
from typing import List
import os
import fitz  # PyMuPDF library
from transformers import AutoTokenizer, AutoModelWithLMHead

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Path to store uploaded files
upload_directory = '/content/uploaded_files'
text_directory = '/content/text_files'


# Home route
@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI server!"}


@app.post("/file")
async def upload_file(file: UploadFile = File(...)):
    # Do here your stuff with the file
    return {"filename": file.filename}

@app.post("/extract-text")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Close the PDF document
        pdf_document.close()

        return JSONResponse(content={"extracted_text": text}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/summarize")
async def extract_text_from_pdf(file: UploadFile = UploadFile(...)):
    try:
        # Read PDF content
        pdf_content = await file.read()

        # Open PDF with PyMuPDF
        pdf_document = fitz.open("pdf", pdf_content)

        # Initialize an empty string to store extracted text
        text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text += page.get_text()

        # Initialize the model and tokenizer
        tokenizer=AutoTokenizer.from_pretrained('T5-base')
        model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)

        # Encode the text
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt",truncation=True)

        # Generate the summary
        outputs = model.generate(inputs,
        max_length=1000, min_length=100)

        # Decode the summary
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return JSONResponse(content={"summary": summary}, status_code=200)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Example Usage:
# Send a PDF file to http://your-fastapi-host/extract-text using your frontend.


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://b6e5-34-85-172-54.ngrok-free.app


INFO:     Started server process [372]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:     49.249.148.147:0 - "POST /summarize HTTP/1.1" 200 OK


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:     49.249.148.147:0 - "POST /summarize HTTP/1.1" 200 OK


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:     203.145.142.86:0 - "POST /summarize HTTP/1.1" 200 OK


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INFO:     203.145.142.86:0 - "POST /summarize HTTP/1.1" 200 OK


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
